In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.5.1/spark-3.5.1-bin-hadoop3.tgz
!tar xf spark-3.5.1-bin-hadoop3.tgz
!pip install -q findspark

In [21]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.1-bin-hadoop3"

In [22]:
import findspark
findspark.init()

In [23]:
from pyspark.sql import SparkSession, functions as F, Window
from pyspark.sql.types import ArrayType, StringType
import re

In [24]:
spark = SparkSession.builder\
        .master("local[*]")\
        .appName("L2_BD_Spark")\
        .config("spark.jars.packages", "com.databricks:spark-xml_2.12:0.14.0")\
        .getOrCreate()

In [25]:
spark.version

'3.5.1'

#Задание

Сформировать отчёт с информацией о 10 наиболее популярных языках программирования по итогам года за период с 2010 по 2020 годы. Отчёт будет отражать динамику изменения популярности языков программирования и представлять собой набор таблиц "топ-10" для каждого года.

Получившийся отчёт сохранить в формате Apache Parquet.

**Загрузка данных**

In [26]:
posts_path = '/content/posts_sample.xml'
prog_lang_path = '/content/programming-languages.csv'

In [27]:
posts_data = spark.read\
.format('xml')\
.options(rowTag='row')\
.load(posts_path)

print("Data")
posts_data.printSchema()

Data
root
 |-- _AcceptedAnswerId: long (nullable = true)
 |-- _AnswerCount: long (nullable = true)
 |-- _Body: string (nullable = true)
 |-- _ClosedDate: timestamp (nullable = true)
 |-- _CommentCount: long (nullable = true)
 |-- _CommunityOwnedDate: timestamp (nullable = true)
 |-- _CreationDate: timestamp (nullable = true)
 |-- _FavoriteCount: long (nullable = true)
 |-- _Id: long (nullable = true)
 |-- _LastActivityDate: timestamp (nullable = true)
 |-- _LastEditDate: timestamp (nullable = true)
 |-- _LastEditorDisplayName: string (nullable = true)
 |-- _LastEditorUserId: long (nullable = true)
 |-- _OwnerDisplayName: string (nullable = true)
 |-- _OwnerUserId: long (nullable = true)
 |-- _ParentId: long (nullable = true)
 |-- _PostTypeId: long (nullable = true)
 |-- _Score: long (nullable = true)
 |-- _Tags: string (nullable = true)
 |-- _Title: string (nullable = true)
 |-- _ViewCount: long (nullable = true)



In [28]:
prog_lang_data = spark.read\
.option("header", True)\
.option("inferSchema", True)\
.option("timestampFormat", 'M/d/y H:m')\
.csv(prog_lang_path)

print("Programming languages")
prog_lang_data.printSchema()

Programming languages
root
 |-- name: string (nullable = true)
 |-- wikipedia_url: string (nullable = true)



In [29]:
# Подготовка списка языков программирования в нижнем регистре
language_names = [row.name.lower() for row in prog_lang_data.select(F.lower("name").alias("name")).collect()]

# Функция для извлечения языков программирования из тегов
def extract_programming_languages(tag_string):
    if not tag_string:
        return []

    # Ищем все теги между <>
    found_tags = re.findall(r'<([^<>]*)>', tag_string)

    # Оставляем только теги, которые есть в нашем списке языков
    return [tag for tag in found_tags if tag in language_names]

# Регистрация UDF функции
extract_langs_udf = F.udf(extract_programming_languages, ArrayType(StringType()))

# Обработка данных постов
processed_posts = (
    posts_data
    # Извлекаем языки из тегов
    .withColumn("languages", extract_langs_udf(F.col("_Tags")))
    # Добавляем год из даты создания
    .withColumn("post_year", F.year("_CreationDate"))
    # Разбиваем массив языков на отдельные строки
    .select(
        "post_year",
        F.explode("languages").alias("language"),
        "_ViewCount"
    )
    # Фильтруем по нужному периоду
    .filter(F.col("post_year").between(2010, 2020))
)

# Агрегация данных по популярности (сумма просмотров)
language_popularity = (
    processed_posts
    .groupBy("post_year", "language")
    .agg(F.sum("_ViewCount").alias("total_views"))
)

# Ранжирование языков по популярности внутри каждого года
window_spec = Window.partitionBy('post_year').orderBy(F.desc('total_views'))
top_10_languages = (
    language_popularity
    # Добавляем позицию в топе
    .withColumn('position', F.row_number().over(window_spec))
    # Оставляем только топ-10
    .filter(F.col('position') <= 10)
)

# Сохранение результатов в Parquet
(
    top_10_languages
    .coalesce(1)
    .write
    .mode('overwrite')
    .format('parquet')
    .save('top_10_languages_per_year.parquet')
)

# Вывод результатов
saved_results = spark.read.parquet('top_10_languages_per_year.parquet')

for year in range(2010, 2021):
    print(f"\nТОП-10 языков программирования за {year} год")
    (
        saved_results
        .filter(F.col("post_year") == year)
        .select("language", "total_views")   # Выводить будем не все, а только необходимое
        .orderBy(F.desc("total_views"))     # Сортируем по просмотрам
        .show(truncate=False)
    )


ТОП-10 языков программирования за 2010 год
+-----------+-----------+
|language   |total_views|
+-----------+-----------+
|php        |1189629    |
|java       |563211     |
|javascript |316131     |
|objective-c|97009      |
|ruby       |76215      |
|c          |66587      |
|python     |60672      |
|matlab     |51865      |
|applescript|32305      |
|delphi     |13065      |
+-----------+-----------+


ТОП-10 языков программирования за 2011 год
+-----------+-----------+
|language   |total_views|
+-----------+-----------+
|javascript |809078     |
|java       |389834     |
|php        |246770     |
|c          |238277     |
|objective-c|218934     |
|python     |203180     |
|bash       |60805      |
|ruby       |39223      |
|perl       |28502      |
|matlab     |18816      |
+-----------+-----------+


ТОП-10 языков программирования за 2012 год
+-----------+-----------+
|language   |total_views|
+-----------+-----------+
|java       |661770     |
|javascript |572750     |
|php    